In [39]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import gensim
import re
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/anaconda3/envs/pythonStudy/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [69]:
import pandas as pd
from tqdm import tqdm

In [65]:
from konlpy.tag import Kkma
k = Kkma()

In [57]:
data = pd.read_csv('/Users/JEONG/Desktop/test2.csv')
data.head(5)

,Unnamed: 0,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트
0,0,201301,카페림보 - 김한민展 :: Exhibit Theater,http://www.arthub.co.kr/sub01/board03_view.htm...,['김한민(Kim Hanmin)'],2013-01-02 ▶ 2013-02-27,2013-02-15 / 16,Open 11:00 ~ Close 23:30,테이크아웃드로잉(Takeout Drawing) 이태원 다른전시 보기,서울시 용산구 이태원동 637,02-790-2637,www.takeoutdrawing.com,[''],● 카페림보 - 김한민展\n 생의 의미와 목적 그리고 사회에 대한 소속...
1,1,201301,박명희展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,['박명희(Park Myunghee)'],2013-01-15 ▶ 2013-01-27,없음,Open 9:00 ~ Close 20:00,세종갤러리(SEJONG GALLERY) 다른전시 보기,서울시 중구 충무로2가 61-3 세종호텔,02-3705-9021,www.sejonggallery.co.kr,[''],● 박명희展\n ‘꿈과 자연’이라는 주제로 시각적인 형상과 상상의 나래...
2,2,201301,잘 어질러진 방 : my place - 유소라展 :: Quilt,http://www.arthub.co.kr/sub01/board03_view.htm...,['유소라(Yu Sora)'],2013-01-30 ▶ 2013-02-19,2013-01-30 PM 5:00,Open 10:00 ~ Close 18:00(일요일 휴관),갤러리41(Gallery41) 다른전시 보기,서울시 종로구 사간동 41,02-744-0341,NaN,[''],● 잘 어질러진 방 : my place\n - 소소한 욕심들로 채워...
3,3,201301,부드러운 교란-백남준을 말하다 :: Media Art,http://www.arthub.co.kr/sub01/board03_view.htm...,"['백남준', '저드 얄커트', '만프레드 레베', '샬럿 무어만']",2013-01-29 ▶ 2013-06-30,2013-02-07 PM 5:00,"Open 10:00 ~ Close 18:00(2주, 4주 월요일 휴관)",백남준아트센터(NJP ART CENTER) 다른전시 보기,경기도 용인시 기흥구 상갈동 85,031-201-8529,www.njpartcenter.kr,[''],● 부드러운 교란-백남준을 말하다\n 백남준아트센터의 2013년도 첫 ...
4,4,201301,space-scape :: Various,http://www.arthub.co.kr/sub01/board03_view.htm...,"['김민정', '김병주', '김용관']",2013-01-05 ▶ 2013-01-28,NaN,Open 10:30 ~ Close 20:00(주말 20:30),롯데갤러리 일산점 다른전시 보기,경기도 고양시 일산동구 장항동 784 롯데백화점 일산점 B1,031-909-2688,NaN,[''],● 세 가지 공간 그리고 하나의 풍경\n 저희 롯데갤러리 일산점에서는 ...


In [58]:
data.fillna('', inplace = True)

In [72]:
corpus_raw = u""
sentences = []
for i,text in enumerate(tqdm(data['텍스트'])) :
    print('Reading "{}" ...'.format(i))
    text = re.sub(r'[^a-zA-Z가-힣]', " ", text)
    text = re.sub(r'[ ]{3,}', " ", text)
    corpus_raw += text
    print("Corpus is now {} characters long".format(len(corpus_raw)))
    sentences.extend(k.sentences(text))
    print()



  0%|          | 0/181 [00:00<?, ?it/s]

Reading "0" ...
Corpus is now 709 characters long




  1%|          | 1/181 [00:00<01:04,  2.79it/s]

  1%|          | 2/181 [00:00<00:44,  4.04it/s]


Reading "1" ...
Corpus is now 1053 characters long

Reading "2" ...
Corpus is now 3078 characters long




  2%|▏         | 3/181 [00:01<01:12,  2.45it/s]

...



 99%|█████████▉| 179/181 [02:38<00:01,  1.13it/s]


Reading "179" ...
Corpus is now 255075 characters long




 99%|█████████▉| 180/181 [02:41<00:00,  1.11it/s]


Reading "180" ...
Corpus is now 255658 characters long




100%|██████████| 181/181 [02:42<00:00,  1.12it/s]



In [73]:
corpus_raw

'  카페림보 김한민 생의 의미와 목적 그리고 사회에 대한 소속감을 철저히 잃어버린  명  림보족 들이 황폐함의 나라 제  국에서 벌이는 게릴라식 저항 과정을 다룬 그래픽 노블  카페림보 를 원작으로 한 창작극이 테이크아웃드로잉 이태원동에서 최초 상연된다 년  월과 월  테이크아웃드로잉 이태원동은  카페림보 가 된다  작가 김한민은 극단장이 되고  무대미술가 이수연은 큐레이터이자 공간연출자가 되며 카페 공간은 연극의 무대로 변신한다  극장의 형식에 구애 받음 없이 자유스러운 연극 공간의 창출과 즉흥적 성격을 강조한 환경연극 의 정신을 이어받아 연극 카페림보 는 무대와 객석  관람방식에 대한 고정된 틀을 깨고자 한다  마치 작품들이 살아 움직이는 상상 속의 미술관에 온 것처럼  관람객들은 도슨트의 안내를 받아 공간에 전시된 작품들을 둘러보다가  그 작품들이 스스로 연극을 벌이는 것을 접하게 되며 미리 고안된 몇몇 장치들에 의해 자연스럽게 연극의 한 부분에 동화되는 기묘한 체험을 하게 된다 그래서  전시 연극 이 이 특별한 연극의 또 다른 이름이다 이 연극적 실험은  카페의 일반 이용객들은 눈길을 끄는 전시물들을 따라 자연스럽게 전시장이자 연극무대로 변화한 공간을 평소보다 조금 더 자세히 관찰하고 조금 더 새롭게 인지하게 되기를  원작을 아는 독자의 경우는 책이 삼차원 공간에 구현되어 어느 순간 책 속에 들어와 있는 것 같은 경험을 하게 되기를 기대한다   박명희 꿈과 자연 이라는 주제로 시각적인 형상과 상상의 나래로 펼쳐진 모습들을 아름다운 색채로 표현해 박명희 작가는 오랜 세월  그림을 통해 작가의 내면을 더욱 아름답게 치장했습니다  그것은 일상의 굴레에서 벗어나 자연이라는 창문을 통해 아름다운 꿈과 영혼의 세계로 떠나는 하나의 여행이기도 합니다  흩날리는 바람처럼 간결하면서도 가녀린 붓터치는 사랑에 빠진 신부의 마음을 대변합니다  밝고 화사한 색채는 앞으로 다가올 밝은 미래에 대한 기대감을 보여주는 장치입니다  박명희 작가의 여행에서 만난 순백의 웨딩드레스를

In [74]:
sentences

['카페 림보 김한 민 생의 의미와 목적 그리고 사회에 대한 소속감을 철저히 잃어버린 명 림보 족 들이 황 폐함의 나라 제 국에서 벌이는 게릴라 식 저항 과정을 다룬 그래픽 노 블 카페 림보 를 원작으로 한 창작극이 테이크 아웃 드로잉 이태원동에서 최초 상연된다 년 월과 월 테이크 아웃 드로잉 이태원동은 카페 림보 가 된다 작가 김한 민은 극단장이 되고 무대 미술가 이수연은 큐레이터 이자 공간 연출자가 되며 카페 공간은 연극의 무대로 변신한다 극장의 형식에 구애 받음 없이 자유스러운 연극 공간의 창출과 즉흥적 성격을 강조한 환경 연극 의 정신을 이어받아 연극 카페 림보 는 무대와 객석 관람 방식에 대한 고정된 틀을 깨고자 한다 마치 작품들이 살아 움직이는 상상 속의 미술관에 온 것처럼 관람객들은 도슨트의 안내를 받아 공간에 전시된 작품들을 둘러보다가 그 작품들이 스스로 연극을 벌이는 것을 접하게 되며 미리 고안된 몇몇 장치들에 의해 자연스럽게 연극의 한 부분에 동화되는 기묘한 체험을 하게 된다',
 '그래서 전시 연극 이 이 특별한 연극의 또 다른 이름이다 이 연극적 실험은 카페의 일반 이용객들은 눈길을 끄는 전시물들을 따라 자연스럽게 전시장 이자 연극무대로 변화한 공간을 평소보다 조금 더 자세히 관찰하고 조금 더 새롭게 인지하게 되기를 원작을 아는 독자의 경우는 책이 삼차원 공간에 구현되어 어느 순간 책 속에 들어와 있는 것 같은 경험을 하게 되기를 기대한다',
 '박 명희 꿈과 자연 이라는 주제로 시각적인 형상과 상상의 나래로 펼쳐진 모습들을 아름다운 색채로 표현해 박 명희 작가는 오랜 세월 그림을 통해 작가의 내면을 더욱 아름답게 치장했습니다',
 '그것은 일상의 굴레에서 벗어 나 자연이라는 창문을 통해 아름다운 꿈과 영혼의 세계로 떠나는 하나의 여행이기도 합니다',
 '흩날리는 바람처럼 간결하면서도 가녀린 붓 터치는 사랑에 빠진 신부의 마음을 대변합니다',
 '밝고 화사한 색채는 앞으로 다가올 밝은 미래에 대한 기대감을 보여주는 장치입니다',
 '박 명

In [77]:
# Take a look at a sentence before and after it is cleaned.
print(sentences[6])

박 명희 작가의 여행에서 만난 순백의 웨딩드레스를 걸친 신부들 작가는 그 신부들을 사랑하고 그들과 삶의 환희를 공유하며 그들을 그리고 있습니다


In [114]:
def sentence_to_wordlist(raw):
#   Remove all characters except letters
    words = list(set(each[0] for each in k.pos(raw) if each[1] not in ['ECE','ESC','ECD','JKG','XSN','XSA','ETD','JX','JKM','JKO','VCP','EFN','EPT'] )) 
    return words

In [115]:
print(sentences[2])
print(sentence_to_wordlist(sentences[2]))

박 명희 꿈과 자연 이라는 주제로 시각적인 형상과 상상의 나래로 펼쳐진 모습들을 아름다운 색채로 표현해 박 명희 작가는 오랜 세월 그림을 통해 작가의 내면을 더욱 아름답게 치장했습니다
['더욱', '세월', '형상', '어', '펼쳐지', '색채', '내면', '치장', '과', '모습', '주제', '나래', '아름', '작가', '꿈', '상상', '박', '오랜', '시각적', '하', '아름답', '자연', '표현', '명희', '통하', '그림']


In [116]:
# Find the total number of tokens in sentences
token_count = sum([len(sentence) for sentence in sentences])
print("The book corpus contains {0:,} tokens".format(token_count))

The book corpus contains 253,452 tokens


## Word2Vec

In [118]:
# Set the parameteres for Word2Vec
num_features = 300
min_word_count = 20
num_workers = multiprocessing.cpu_count()
context_size = 10
downsampling = 1e-4
seed = 2

In [119]:
texts2vec = w2v.Word2Vec(
    sg=1, #skip-gram
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [120]:
# Build the vocabulary
texts2vec.build_vocab(sentences)
print("texts2vec vocabulary length:", len(texts2vec.wv.vocab))

texts2vec vocabulary length: 579


In [129]:
texts2vec.train(sentences, total_examples= (len(sentences)), epochs=5)

(260274, 1267260)

In [134]:
texts2vec.wv.most_similar('형')

[('각', 0.3265348970890045),
 ('조', 0.30985844135284424),
 ('가', 0.29208076000213623),
 ('강', 0.2849295139312744),
 ('작', 0.28413572907447815),
 ('현', 0.28374990820884705),
 ('시', 0.28217530250549316),
 ('색', 0.2799277603626251),
 ('식', 0.2676527500152588),
 ('서', 0.26680219173431396)]